## 네이버 뉴스 댓글 수집 함수 생성

- 네이버뉴스 링크가 `https://n.news`로 시작하는 댓글만 수집 가능

### 데이터 준비

In [ ]:
# 관련 라이브러리를 호출합니다.
import os
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as BTS
import json
import re
import time
from tqdm.notebook import tqdm

In [ ]:
# 현재 작업 경로를 확인합니다.
os.getcwd()

In [ ]:
# data 폴더로 작업 경로를 변경합니다.
os.chdir(path = '../data')

In [ ]:
# 현재 작업 경로에 있는 폴더명과 파일명을 확인합니다.
sorted(os.listdir())

In [ ]:
# pkl 파일을 읽고 newsLinks를 생성합니다.
newsLinks = pd.read_pickle(filepath_or_buffer = 'Naver_News_Link.pkl')

### 함수 생성

In [ ]:
# 네이버뉴스 링크로 뉴스 댓글 개수 및 목록을 수집하는 함수를 생성합니다.
def NaverNewsReply(nlink, pageSize = 100):
    
    # nlink에 따라 HTTP 요청 실행 및 응답 바디 문자열을 처리하는 코드를 분기합니다.
    if 'https://n.news' in nlink:
        
        # 요청 URL을 설정합니다.
        url = 'https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json'
        
        # nlink에서 cid(company id)와 aid(article id)를 추출합니다.
        cid = re.findall(pattern = r'(?<=article/)(\d+)', string = nlink)[0]
        aid = re.findall(pattern = r'(?<=\d/)(\d+$)', string = nlink)[0]
        
        # 쿼리 문자열을 설정합니다.
        query = {
            'ticket': 'news',
            'pool': 'cbox5', 
            'lang': 'ko', 
            'country': 'KR', 
            'objectId': f'news{cid},{aid}', 
            'pageSize': pageSize, 
            'indexSize': 10, 
            'listType': 'OBJECT', 
            'pageType': 'more', 
            'page': 1, 
            'sort': 'favorite', 
            'includeAllStatus': 'true'
        }
        
        # 요청 헤더를 설정합니다.
        # [참고] referer에 nlink를 지정합니다.
        headers = {
            'content-type': 'application/javascript;charset=UTF-8', 
            'referer': nlink, 
            'user-agent': 'Mozilla/5.0'
        }
        
        # HTTP 요청을 실행합니다.
        res = requests.get(url = url, params = query, headers = headers)
        
        # HTTP 응답 바디 문자열에서 중괄호 앞에 있는 불필요한 문자열을 삭제합니다.
        # [참고] str.replace()는 시리즈 원소에서 지정한 패턴을 변경(또는 삭제)합니다.
        text = re.sub(pattern = r'jQuery.+?\(|_callback\(|\);$', repl = '', string = res.text)
        
        # JSON 형태의 문자열을 딕셔너리로 변환합니다.
        dat = json.loads(s = text)
        
        # 뉴스 댓글 개수를 replyCount에 할당합니다.
        replyCount = dat['result']['count']['comment']
        
        # 뉴스 댓글 개수가 0이면 None으로 결과를 반환합니다.
        if replyCount == 0:
            return {'replyCount': 0, 'totalPages': 0, 'replyList': None}
        
        else:
            # 뉴스 총 페이지수를 totalPages에 할당합니다.
            totalPages = dat['result']['pageModel']['totalPages']
            
            # 1페이지 뉴스 댓글 목록을 replyPage1에 할당합니다.
            replyPage1 = pd.DataFrame(data = dat['result']['commentList'])
            
            # replyPage1에서 선택할 일부 열이름을 리스트로 생성합니다.
            cols = ['objectId', 'commentNo', 'parentCommentNo', 'replyAllCount', 'contents', 'userName', 
                    'modTime', 'regTime', 'sympathyCount', 'antipathyCount', 'hiddenByCleanbot', 'deleted']
            
            # 총 페이지수가 1이면 replyPage1을 반환하고 총 페이지수가 2 이상이면 for 반복문을 실행합니다.
            if totalPages == 1:
                
                # 결과를 반환합니다.
                return {'replyCount': replyCount, 'totalPages': totalPages, 'replyList': replyPage1[cols]}
            
            elif totalPages >= 2:
                
                # 반복문 실행 범위는 총 페이지수에 1을 더한 값으로 설정합니다.
                for page in range(2, totalPages + 1):
                    
                    # 다음 페이지 수집에 필요한 쿼리 문자열을 more에 할당합니다.
                    more = dat['result']['morePage']
                    
                    # 쿼리 문자열을 설정합니다.
                    query = {
                        'ticket': 'news',
                        'pool': 'cbox5', 
                        'lang': 'ko', 
                        'country': 'KR', 
                        'objectId': f'news{cid},{aid}', 
                        'pageSize': 20, 
                        'indexSize': 10, 
                        'listType': 'OBJECT', 
                        'pageType': 'more', 
                        'page': page, 
                        'sort': 'favorite', 
                        'moreParam.direction': 'next', 
                        'moreParam.prev': more['prev'], 
                        'moreParam.next': more['next'], 
                        'includeAllStatus': 'true'
                    }
                    
                    # HTTP 요청을 실행합니다.
                    res = requests.get(url = url, params = query, headers = headers)
                    
                    # HTTP 응답 바디 문자열에서 중괄호 앞에 있는 불필요한 문자열을 삭제합니다.
                    text = re.sub(pattern = r'jQuery.+?\(|_callback\(|\);$', repl = '', string = res.text)
                    
                    # JSON 형태의 문자열을 딕셔너리로 변환합니다.
                    dat = json.loads(s = text)
                    
                    # 2페이지 이후 뉴스 댓글 목록을 replyPage2에 할당합니다.
                    replyPage2 = pd.DataFrame(data = dat['result']['commentList'])
                    
                    # replyPage2을 replyPage1에 행 방향으로 결합하고 replyPage1에 재할당합니다.
                    replyPage1 = pd.concat(objs = [replyPage1, replyPage2], ignore_index = True)
                
                # 결과를 반환합니다.
                return {'replyCount': replyCount, 'totalPages': totalPages, 'replyList': replyPage1[cols]}
    
    # nlink가 'https://n.news'로 시작하지 않으면 None으로 결과를 반환합니다.
    else:
        return {'replyCount': 0, 'totalPages': 0, 'replyList': None}

### 데이터 수집

In [ ]:
# 경고를 출력하지 않도록 설정합니다.
import warnings
warnings.filterwarnings(action = 'ignore')

In [ ]:
# 반복 실행할 횟수를 n에 할당합니다.
n = len(newsLinks)

# 뉴스 댓글 개수를 저장할 시리즈를 생성합니다.
newsRpCnt = pd.Series(data = [np.nan] * n, dtype = float)

# 뉴스 댓글 본문을 저장할 빈 데이터프레임을 생성합니다.
newsReply = pd.DataFrame()

# for 반복문으로 네이버뉴스 댓글 개수 및 목록을 수집합니다.
for i in tqdm(range(n)):
    
    # 반복문 실행 도중 에러가 발생하면 다음 원소(링크)로 건너뛰도록 설정합니다.
    try:
        
        # 네이버뉴스 링크로 뉴스 댓글 개수 및 목록을 수집하는 함수를 실행하고 result에 할당합니다.
        result = NaverNewsReply(nlink = newsLinks.iloc[i])
        
        # 뉴스 댓글 개수를 수집하고 newsRpCnt에 i번째 원소로 할당합니다.
        newsRpCnt.iloc[i] = result['replyCount']
        
        # 뉴스 댓글 개수가 1 이상이면 newsReply에 행 방향으로 추가하고 0이면 다음 원소(링크)를 실행합니다.
        if result['replyCount'] >= 1:
            
            # 뉴스 댓글 목록을 수집하고 newsReply에 행 방향으로 추가합니다.
            newsReply = pd.concat(objs = [newsReply, result['replyList']], ignore_index = True)
        
        else:
            pass

    # 반복문 실행 도중 에러가 발생하면 다음 원소(링크)로 건너뜁니다.
    except:
        next
    
    # 1초간 멈춥니다.
    time.sleep(1)

In [ ]:
# newsRpCnt의 처음 5행을 확인합니다.
newsRpCnt.head()

In [ ]:
# newsReply의 처음 10행을 확인합니다.
newsReply.head(n = 10)

In [ ]:
# newsReply의 정보 확인
newsReply.info()

### 외부 파일로 저장

In [ ]:
# 현재 작업 경로를 확인합니다.
os.getcwd()

In [ ]:
# newsRpCnt를 pkl 파일로 저장합니다.
pd.to_pickle(obj = newsRpCnt, filepath_or_buffer = 'Naver_News_ReCnt.pkl')

In [ ]:
# newsReply를 pkl 파일로 저장합니다.
pd.to_pickle(obj = newsReply, filepath_or_buffer = 'Naver_News_Reply.pkl')

In [ ]:
# 현재 작업 경로에 있는 폴더명과 파일명을 확인합니다.
sorted(os.listdir())

## End of Document